# 模型訓練 (Model Training)
## 第三期大腸癌存活預測研究

本筆記本訓練多種存活預測模型

In [ ]:
# 導入套件
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 設定路徑
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# 導入自訂模組
from src.model_training import SurvivalModelTrainer
from src.utils import load_config

print("套件載入完成")

In [ ]:
# 載入配置
config = load_config(str(project_root / 'config' / 'config.yaml'))

# 載入特徵工程後的資料
train_df = pd.read_csv(project_root / 'data' / 'processed' / 'train_features.csv')
test_df = pd.read_csv(project_root / 'data' / 'processed' / 'test_features.csv')

print(f"訓練集形狀: {train_df.shape}")
print(f"測試集形狀: {test_df.shape}")

In [ ]:
# 準備資料
# 根據實際欄位名稱調整
duration_col = 'survival_time'  # 存活時間欄位
event_col = 'event'  # 事件欄位 (0=censored, 1=event)

# 檢查必要欄位是否存在
if duration_col in train_df.columns and event_col in train_df.columns:
    print(f"存活時間欄位: {duration_col}")
    print(f"事件欄位: {event_col}")
    print(f"\n事件統計:")
    print(train_df[event_col].value_counts())
else:
    print("警告: 找不到存活時間或事件欄位，請檢查資料")

In [ ]:
# 初始化訓練器
trainer = SurvivalModelTrainer(config)

print("模型訓練器已初始化")

## 1. Cox 比例風險模型

In [ ]:
# 訓練 Cox 比例風險模型
if duration_col in train_df.columns and event_col in train_df.columns:
    try:
        cox_model = trainer.train_cox_ph(
            train_df,
            duration_col=duration_col,
            event_col=event_col
        )
        print("\nCox 模型訓練成功！")
    except Exception as e:
        print(f"Cox 模型訓練失敗: {e}")
else:
    print("跳過 Cox 模型訓練")

## 2. 隨機存活森林

In [ ]:
# 準備資料給隨機存活森林
if duration_col in train_df.columns and event_col in train_df.columns:
    try:
        from sksurv.util import Surv
        
        # 準備特徵和目標
        feature_cols = [col for col in train_df.columns 
                       if col not in [duration_col, event_col]]
        
        X_train = train_df[feature_cols].values
        y_train = Surv.from_dataframe(event_col, duration_col, train_df)
        
        # 訓練模型
        rsf_model = trainer.train_random_survival_forest(
            X_train,
            y_train,
            n_estimators=config['models']['random_forest']['n_estimators'],
            random_state=config['models']['random_forest']['random_state']
        )
        
        print("\n隨機存活森林訓練成功！")
        
    except ImportError:
        print("警告: 未安裝 scikit-survival，跳過隨機存活森林")
    except Exception as e:
        print(f"隨機存活森林訓練失敗: {e}")
else:
    print("跳過隨機存活森林訓練")

## 3. 其他模型 (可選)

In [ ]:
# 可以在這裡添加其他模型
# 例如: XGBoost, LightGBM, Deep Learning 等

print("可以根據需求添加更多模型")

## 儲存模型

In [ ]:
# 儲存所有訓練好的模型
models_dir = project_root / 'models'
models_dir.mkdir(exist_ok=True)

trainer.save_all_models(str(models_dir))

print("\n所有模型已儲存")
print(f"儲存位置: {models_dir}")
print("\n模型訓練完成！")

In [ ]:
# 顯示已訓練的模型
print("已訓練的模型:")
for model_name in trainer.models.keys():
    print(f"  - {model_name}")